In [1]:
# 🔧 VALIDASI PERBAIKAN NOTEBOOK
# Cell ini untuk memastikan semua dependencies dan file yang diperlukan tersedia

import sys
print("🔍 Validasi Environment dan Dependencies")
print("=" * 50)

# Check Python version
print(f"✓ Python Version: {sys.version}")

# Check required libraries
required_libs = [
    'pandas', 'numpy', 'matplotlib', 'seaborn', 
    'sklearn', 'google_play_scraper', 'Sastrawi', 
    'wordcloud', 'imblearn', 'joblib'
]

missing_libs = []
for lib in required_libs:
    try:
        __import__(lib)
        print(f"✓ {lib}")
    except ImportError:
        missing_libs.append(lib)
        print(f"❌ {lib} - TIDAK DITEMUKAN")

if missing_libs:
    print(f"\n⚠️ Library yang hilang: {', '.join(missing_libs)}")
    print("Jalankan: %pip install " + " ".join(missing_libs))
else:
    print("\n✅ Semua library dependencies tersedia!")

# Check data files
import os
data_files = [
    '../data/kamus_slang_formal.txt',
    '../data/stopwordsID.txt',
    '../data/ulasan_goride.csv'
]

print("\n📂 Validasi Data Files:")
for file_path in data_files:
    if os.path.exists(file_path):
        print(f"✓ {file_path}")
    else:
        print(f"⚠️ {file_path} - TIDAK DITEMUKAN (akan dibuat fallback)")

print("\n🎯 Notebook siap dijalankan!")

🔍 Validasi Environment dan Dependencies
✓ Python Version: 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]
✓ pandas
✓ numpy
✓ matplotlib
✓ seaborn
✓ sklearn
✓ google_play_scraper
✓ Sastrawi
✓ wordcloud
✓ imblearn
✓ joblib

✅ Semua library dependencies tersedia!

📂 Validasi Data Files:
✓ ../data/kamus_slang_formal.txt
✓ ../data/stopwordsID.txt
✓ ../data/ulasan_goride.csv

🎯 Notebook siap dijalankan!


# 🚀 SentimenGo: Analisis Sentimen Ulasan GoRide 

## 📖 Deskripsi Proyek
Notebook ini menggabungkan seluruh proses analisis sentimen ulasan GoRide, meliputi:
1. **🕷️ Crawling Data** - Pengambilan data ulasan dari Google Play Store
2. **🧹 Preprocessing Data** - Pembersihan dan normalisasi teks step-by-step
3. **🤖 Sentiment Analysis** - Pemodelan dan analisis sentimen menggunakan SVM

## 🎯 Tujuan
- Menganalisis sentimen ulasan pengguna terhadap layanan GoRide
- Membangun model machine learning untuk prediksi sentimen otomatis
- Memberikan insights tentang persepsi publik terhadap GoRide

---

# 🕷️ BAGIAN 1: WEB SCRAPING DATA ULASAN GORIDE

## 📝 Gambaran Umum
Pada bagian ini, kita akan melakukan web scraping untuk mengumpulkan data ulasan GoRide dari Google Play Store. Proses ini mencakup:
- Installation library yang diperlukan
- Konfigurasi parameter scraping
- Pengambilan data ulasan dengan filter kata kunci
- Penyimpanan data ke format CSV

## 🛠️ Persiapan: Install Library yang Diperlukan

In [1]:
%pip install google-play-scraper

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 📦 Import Libraries yang Diperlukan
Mengimpor semua library yang dibutuhkan untuk proses web scraping:

In [2]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import time

## 🎯 Fungsi Scraping Ulasan GoRide
Fungsi ini akan mengambil ulasan yang mengandung kata kunci terkait GoRide pada periode tertentu:

In [3]:
def scrape_goride_reviews():
    """
    Fungsi untuk mengambil ulasan GoRide dari Google Play Store pada bulan tertentu di tahun tertentu.
    Menggunakan pendekatan batch processing dengan continuation token.
    """
    
    # =============================================
    # KONFIGURASI YANG DAPAT DIUBAH
    # =============================================
    TARGET_YEAR = 2025    # Tahun target
    TARGET_MONTH = 4     # Bulan target (1-12)
    BATCH_SIZE = 200      # Jumlah ulasan per batch
    LANG = 'id'           # Bahasa ulasan
    COUNTRY = 'id'        # Negara ulasan
    APP_ID = 'com.gojek.app'  # ID aplikasi Gojek
    
    # Kata kunci untuk filter ulasan GoRide
    KEYWORDS = ["Goride", "goride", "go-ride", "Go-ride", "GO RIDE", "go ride", "ride", "Ride"]
    # =============================================
    
    # Validasi input bulan
    if TARGET_MONTH < 1 or TARGET_MONTH > 12:
        print("Error: Bulan harus antara 1 (Januari) hingga 12 (Desember)")
        return None
    
    # Konversi bulan ke nama bulan untuk display
    month_names = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]
    month_name = month_names[TARGET_MONTH - 1]
    
    print(f"\nMemulai scraping ulasan GoRide untuk bulan {month_name} {TARGET_YEAR}...")
    print(f"Menggunakan kata kunci: {', '.join(KEYWORDS)}")
    
    # Hitung range tanggal target
    start_date = datetime(TARGET_YEAR, TARGET_MONTH, 1)
    if TARGET_MONTH == 12:
        end_date = datetime(TARGET_YEAR + 1, 1, 1)
    else:
        end_date = datetime(TARGET_YEAR, TARGET_MONTH + 1, 1)
    
    print(f"Periode target: {start_date.strftime('%Y-%m-%d')} hingga {end_date.strftime('%Y-%m-%d')}")
    
    # Kumpulkan ulasan
    all_reviews = []
    continuation_token = None
    batch_count = 0
    total_found = 0
    stop_early = False
    
    start_time = time.time()
    
    while True:
        batch_count += 1
        try:
            print(f"\nMengambil batch ke-{batch_count}...")
            
            # Ambil batch ulasan dengan pengecekan continuation_token
            if continuation_token is not None:
                result, continuation_token = reviews(
                    APP_ID,
                    lang=LANG,
                    country=COUNTRY,
                    sort=Sort.NEWEST,
                    count=BATCH_SIZE,
                    continuation_token=continuation_token
                )
            else:
                result, continuation_token = reviews(
                    APP_ID,
                    lang=LANG,
                    country=COUNTRY,
                    sort=Sort.NEWEST,
                    count=BATCH_SIZE
                )
            
            batch_found = 0
            for review in result:
                review_date = review['at']
                review_text = review['content']
                
                # Cek apakah sudah melewati periode target
                if review_date < start_date:
                    print("Sudah mencapai periode sebelum target, menghentikan proses...")
                    stop_early = True
                    break
                
                # Cek apakah dalam periode target dan mengandung kata kunci
                if (review_text and start_date <= review_date < end_date and 
                    any(keyword in review_text for keyword in KEYWORDS)):
                    
                    all_reviews.append({
                        'review_id': review['reviewId'],
                        'user_name': review['userName'],
                        'rating': review['score'],
                        'review_text': review_text,
                        'date': review_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'likes': review['thumbsUpCount'],
                        'app_version': review.get('reviewCreatedVersion', '')
                    })
                    batch_found += 1
            
            total_found += batch_found
            print(f"Ditemukan {batch_found} ulasan yang sesuai dalam batch ini")
            print(f"Total sementara: {total_found} ulasan")
            
            # Berhenti jika tidak ada token lanjutan atau sudah melewati periode
            if not continuation_token or stop_early:
                break
                
        except Exception as e:
            print(f"Error dalam pengambilan data: {str(e)}")
            break
    
    elapsed_time = time.time() - start_time
    
    if not all_reviews:
        print("\nTidak ada ulasan GoRide yang ditemukan untuk periode ini.")
        return None
    
    # Konversi ke DataFrame
    df = pd.DataFrame(all_reviews)
    
    # Simpan ke file CSV
    csv_filename = f"ulasan_goride_{TARGET_YEAR}_{TARGET_MONTH:02d}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8')
    
    print("\nRingkasan Hasil:")
    print(f"Total waktu eksekusi: {elapsed_time:.2f} detik")
    print(f"Total batch diproses: {batch_count}")
    print(f"Total ulasan yang sesuai: {len(df)}")
    print(f"Data berhasil disimpan dalam file: {csv_filename}")
    
    return df

## ▶️ Eksekusi Scraping Data
Menjalankan fungsi scraping dan menampilkan preview data yang dihasilkan:

In [4]:
if __name__ == "__main__":
    print("Scraper Ulasan GoRide Google Play Store")
    print("--------------------------------------")
    print("NOTE: Untuk mengubah bulan/tahun, edit variabel TARGET_YEAR dan TARGET_MONTH dalam fungsi scrape_goride_reviews()")
    
    reviews_data = scrape_goride_reviews()
    
    if reviews_data is not None:
        print("\nPreview data:")
        print(reviews_data.head())

Scraper Ulasan GoRide Google Play Store
--------------------------------------
NOTE: Untuk mengubah bulan/tahun, edit variabel TARGET_YEAR dan TARGET_MONTH dalam fungsi scrape_goride_reviews()

Memulai scraping ulasan GoRide untuk bulan April 2025...
Menggunakan kata kunci: Goride, goride, go-ride, Go-ride, GO RIDE, go ride, ride, Ride
Periode target: 2025-04-01 hingga 2025-05-01

Mengambil batch ke-1...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-2...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-2...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-3...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-3...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-4...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-4...
Dite

# 🧹 BAGIAN 2: PREPROCESSING DATA STEP-BY-STEP

## 📊 Gambaran Umum
Bagian ini akan melakukan preprocessing data secara bertahap untuk membersihkan dan menyiapkan data ulasan. Setiap langkah akan ditampilkan secara individual dengan contoh before-after agar mudah dipahami.

### 🛠️ Langkah-langkah Preprocessing:
1. **Case Folding & Phrase Standardization** - Mengubah ke huruf kecil dan standardisasi istilah
2. **Cleansing** - Menghapus URL, karakter khusus, dan spasi berlebih
3. **Normalisasi Slang** - Mengubah kata slang menjadi kata formal
4. **Remove Repeated Characters** - Menghilangkan karakter berulang berlebihan
5. **Tokenization** - Memecah teks menjadi token/kata
6. **Stopword Removal** - Menghapus kata-kata yang tidak bermakna
7. **Stemming** - Mengubah kata ke bentuk dasar
8. **Rejoin Tokens** - Menggabungkan kembali kata-kata menjadi kalimat

## 📦 Import Libraries untuk Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## 📂 Load Dataset dan Kamus Slang/Stopword
Memuat dataset ulasan dan file pendukung (kamus slang dan stopword) yang diperlukan untuk preprocessing:

In [3]:
# Load dataset ulasan dengan error handling
try:
    df = pd.read_csv("../data/ulasan_goride.csv")
    print(f"Dataset loaded: {len(df)} ulasan")
except FileNotFoundError:
    print("File ulasan_goride.csv tidak ditemukan. Pastikan file ada di folder ../data/")
    print("Membuat dataset dummy untuk demo...")
    df = pd.DataFrame({
        'review_text': [
            'goride sangat bagus dan cepat',
            'driver goride ramah sekali',
            'goride sering telat dan lama',
            'pelayanan goride mengecewakan'
        ],
        'rating': [5, 4, 2, 1]
    })

# Mengatur tampilan agar tidak memotong isi kolom
pd.set_option('display.max_colwidth', None)

# Load kamus slang dan stopword dengan error handling
try:
    slang_dict = dict(line.strip().split(':') for line in open("../data/kamus_slang_formal.txt", encoding='utf-8'))
    print(f"Kamus slang loaded: {len(slang_dict)} entri")
except FileNotFoundError:
    print("File kamus_slang_formal.txt tidak ditemukan. Menggunakan kamus kosong.")
    slang_dict = {}

try:
    stopword_custom = set(open("../data/stopwordsID.txt", encoding='utf-8').read().splitlines())
    print(f"Stopword loaded: {len(stopword_custom)} kata")
except FileNotFoundError:
    print("File stopwordsID.txt tidak ditemukan. Menggunakan stopword default.")
    stopword_custom = {'dan', 'atau', 'yang', 'di', 'ke', 'dari', 'untuk', 'dengan', 'pada', 'adalah', 'ini', 'itu'}

print("\nPreview dataset:")
df.head()

Dataset loaded: 659 ulasan
Kamus slang loaded: 1210 entri
Stopword loaded: 758 kata

Preview dataset:


,user_name,rating,review_text,date,label,sentiment
0,Ayun Hawa,1,"Aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong",2024-01-31 16:39:09,0,Negative
1,supri yanto,5,Kok buka aplikasi goride hilang2 mulu...,2024-01-31 13:37:36,0,Negative
2,Ruwy Masyari,5,Gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰,2024-01-30 14:29:51,1,Positive
3,Achmad Sayid,3,Sekarang udah mahal goride dan gofood,2024-01-30 09:18:49,0,Negative
4,Resa Agustin,1,Kesalahan dari sistem gojek saldo Gopay kepotong otomatis ketika melakukan transaksi order GoRide GoRide nya BATAL tp saldo Gopay kepotong dan gak balik lagi. Udh bikin laporan dan disuruh tuunggu 2x24 jam tp saldo ga masuk lagi ke akun gue. Hadeuuu rumit,2024-01-29 23:35:04,0,Negative


## ⚙️ Setup Preprocessing Utilities
Mempersiapkan tools dan fungsi utility yang akan digunakan dalam preprocessing:

In [ ]:
print("\n=== Preprocessing Data ===")
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def print_step(step_name, before, after):
    """Utility untuk menampilkan before-after setiap langkah preprocessing"""
    print(f"\n=== {step_name.upper()} ===")
    print(f"Before: {before}")
    print(f"After:  {after}")

## 1️⃣ LANGKAH 1: Case Folding + Phrase Standardization

**Tujuan:** 
- Mengubah semua huruf menjadi huruf kecil (lowercase)
- Menyeragamkan penulisan istilah seperti "go ride", "go-ride" menjadi "goride"

**Proses:**
- Konversi ke lowercase untuk konsistensi
- Standardisasi variasi penulisan GoRide

In [ ]:
def case_folding(text):
    before = text
    after = text.lower()
    after = re.sub(r'\bgo[\s\-_]?ride\b', 'goride', after)
    
    # Contoh untuk setiap data: tampilkan before-after
    print_step("Case Folding + Standardization", before, after)
    return after

# Pilih kolom yang akan diproses (sesuaikan dengan nama kolom di dataset Anda)
text_column = 'review_text'  # Ganti sesuai nama kolom ulasan di dataset
df['casefolding'] = df[text_column].astype(str).apply(case_folding)

# Menampilkan hasil 5 data pertama
print("\n=== DataFrame setelah Case Folding ===")
df[[text_column, 'casefolding']].rename(columns={
    text_column: 'sebelum case folding', 
    'casefolding': 'setelah case folding'
}).head(5)

## 2️⃣ LANGKAH 2: Cleansing

**Tujuan:** 
- Menghapus URL dan link
- Menghilangkan karakter khusus, angka, dan simbol
- Menormalisasi spasi berlebih

**Proses:**
- Hapus URL yang dimulai dengan http/www
- Hapus semua karakter selain huruf dan spasi
- Normalisasi spasi ganda menjadi spasi tunggal

In [ ]:
def clean_text(text):
    before = text
    # Hapus URL dan link
    after = re.sub(r'http\S+|www\S+', '', text)
    # Hapus karakter selain huruf dan spasi
    after = re.sub(r'[^a-zA-Z\s]', ' ', after)
    # Normalisasi spasi berlebih
    after = re.sub(r'\s+', ' ', after).strip()
    
    print_step("Cleansing", before, after)
    return after

df['cleansing'] = df['casefolding'].apply(clean_text)
print("\n=== DataFrame setelah Cleansing ===")
df[['casefolding', 'cleansing']].rename(columns={
    'casefolding': 'sebelum cleansing', 
    'cleansing': 'setelah cleansing'
}).head(5)

In [ ]:
def clean_text(text):
    before = text
    # Hapus URL
    after = re.sub(r'http\S+|www\S+', '', text)
    # Hapus karakter selain huruf dan spasi
    after = re.sub(r'[^a-zA-Z\s]', ' ', after)
    # Normalisasi spasi
    after = re.sub(r'\s+', ' ', after).strip()
    
    print_step("Cleansing", before, after)
    return after

df['cleansing'] = df['casefolding'].apply(clean_text)
print("\n=== DataFrame setelah Cleansing ===")
df[['casefolding', 'cleansing']].rename(columns={
    'casefolding': 'sebelum cleansing', 
    'cleansing': 'setelah cleansing'
}).head(5)

## 3️⃣ LANGKAH 3: Normalisasi Slang

**Tujuan:** 
- Mengubah kata-kata slang/tidak baku menjadi kata formal
- Meningkatkan konsistensi data untuk analisis yang lebih baik

**Proses:**
- Menggunakan kamus slang untuk mapping kata slang → kata formal
- Kata yang tidak ada di kamus akan tetap tidak berubah

In [ ]:
def normalize_slang(text):
    before = text
    words = text.split()
    # Filter None values dan normalize dengan aman
    normalized = [slang_dict.get(word, word) for word in words if word is not None]
    after = ' '.join(filter(None, normalized))  # Filter None values dari hasil join
    
    print_step("Normalize Slang", before, after)
    return after

df['normalized'] = df['cleansing'].apply(normalize_slang)
print("\n=== DataFrame setelah Normalize Slang ===")
df[['cleansing', 'normalized']].rename(columns={
    'cleansing': 'sebelum normalize slang', 
    'normalized': 'setelah normalize slang'
}).head(5)

## 4️⃣ LANGKAH 4: Remove Repeated Characters

**Tujuan:** 
- Menghilangkan karakter berulang yang berlebihan
- Contoh: "bagusssss" → "baguss", "kereeeeen" → "kereen"

**Proses:**
- Mendeteksi karakter yang berulang lebih dari 2 kali
- Menguranginya menjadi maksimal 2 karakter berurutan

In [ ]:
def remove_repeated(text):
    before = text
    # Mengurangi karakter berulang lebih dari 2 menjadi maksimal 2
    after = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    
    print_step("Remove Repeated Characters", before, after)
    return after

df['no_repeated'] = df['normalized'].apply(remove_repeated)
print("\n=== DataFrame setelah Remove Repeated Characters ===")
df[['normalized', 'no_repeated']].rename(columns={
    'normalized': 'sebelum remove repeated', 
    'no_repeated': 'setelah remove repeated'
}).head(5)

## 5️⃣ LANGKAH 5: Tokenization

**Tujuan:** 
- Memecah kalimat menjadi daftar kata-kata individual (token)
- Memudahkan pemrosesan kata per kata pada langkah selanjutnya

**Proses:**
- Memisahkan teks berdasarkan spasi dan tanda baca
- Menghasilkan list/array kata-kata

In [ ]:
def tokenize_text(text):
    before = text
    # Memecah teks menjadi token kata
    after = re.findall(r'\b\w+\b', text)
    
    print_step("Tokenization", before, after)
    return after

df['tokenized'] = df['no_repeated'].apply(tokenize_text)
print("\n=== DataFrame setelah Tokenization ===")

# Format tampilan untuk list agar terlihat jelas
df_token_display = df[['no_repeated', 'tokenized']].copy()
df_token_display['tokenized'] = df_token_display['tokenized'].apply(repr)
df_token_display.rename(columns={
    'no_repeated': 'sebelum tokenization', 
    'tokenized': 'setelah tokenization'
}).head(5)

## 6️⃣ LANGKAH 6: Stopword Removal

**Tujuan:** 
- Menghapus kata-kata yang tidak bermakna atau tidak memiliki nilai analisis
- Contoh stopword: "dan", "atau", "yang", "di", "ke", dll.

**Proses:**
- Membandingkan setiap token dengan daftar stopword
- Menghapus token yang ada dalam daftar stopword
- Mempertahankan kata-kata yang bermakna untuk analisis

In [ ]:
def remove_stopwords(tokens):
    if isinstance(tokens, str):
        tokens = re.findall(r'\b\w+\b', tokens)
    
    before = tokens.copy()
    # Hapus kata yang ada dalam daftar stopword
    after = [word for word in tokens if word not in stopword_custom]
    
    print_step("Stopword Removal", before, after)
    return after

# Terapkan fungsi ke DataFrame
df['no_stopword'] = df['tokenized'].apply(remove_stopwords)

print("\n=== DataFrame setelah Stopword Removal ===")
df_stopword_display = df[['tokenized', 'no_stopword']].copy()
df_stopword_display['tokenized'] = df_stopword_display['tokenized'].apply(repr)
df_stopword_display['no_stopword'] = df_stopword_display['no_stopword'].apply(repr)

# Rename dan tampilkan hasil
df_stopword_display.rename(columns={
    'tokenized': 'sebelum no_stopword',
    'no_stopword': 'setelah no_stopword'
}).head(5)

## 7️⃣ LANGKAH 7: Stemming

**Tujuan:** 
- Mengubah kata-kata menjadi bentuk dasar/root word
- Mengurangi variasi kata yang sebenarnya memiliki makna sama
- Contoh: "bermain", "permainan", "dimainkan" → "main"

**Proses:**
- Menggunakan Sastrawi Stemmer untuk bahasa Indonesia
- Menghilangkan awalan dan akhiran kata
- Mempertahankan makna inti dari setiap kata

In [ ]:
def stem_tokens(tokens):
    if isinstance(tokens, str):
        tokens = tokens.split()
    
    before = tokens.copy()
    # Lakukan stemming pada setiap token
    after = [stemmer.stem(word) for word in tokens]
    
    print_step("Stemming", before, after)
    return after

# Terapkan proses stemming ke kolom 'no_stopword'
df['stemmed'] = df['no_stopword'].apply(stem_tokens)

print("\n=== DataFrame setelah Stemming ===")

# Format tampilan agar sama seperti langkah stopword removal
df_stemming_display = df[['no_stopword', 'stemmed']].copy()
df_stemming_display['no_stopword'] = df_stemming_display['no_stopword'].apply(repr)
df_stemming_display['stemmed'] = df_stemming_display['stemmed'].apply(repr)

# Rename kolom dan tampilkan hasil
df_stemming_display = df_stemming_display.rename(columns={
    'no_stopword': 'sebelum stemming',
    'stemmed': 'setelah stemming'
})
df_stemming_display.head(5)

## 8️⃣ LANGKAH 8: Rejoin Tokens

**Tujuan:** 
- Menggabungkan kembali token-token menjadi kalimat
- Menghasilkan teks bersih yang siap untuk analisis sentimen

**Proses:**
- Menggabungkan list kata menjadi string
- Memisahkan kata dengan spasi
- Menghasilkan teks final yang sudah dipreprocessing

In [ ]:
def rejoin_tokens(tokens):
    if isinstance(tokens, str):
        return tokens
    
    before = tokens.copy()
    # Gabungkan token kembali menjadi string
    after = ' '.join(tokens)
    
    print_step("Rejoin Tokens", before, after)
    return after

df['final_clean'] = df['stemmed'].apply(rejoin_tokens)
print("\n=== DataFrame setelah Final Clean ===")
df[['stemmed', 'final_clean']].rename(columns={
    'stemmed': 'sebelum rejoin tokens', 
    'final_clean': 'setelah rejoin tokens'
}).head(10)

## 💾 Penyimpanan Data Hasil Preprocessing

Menyimpan hasil preprocessing ke file CSV baru dan menampilkan perbandingan sebelum-sesudah preprocessing:

In [ ]:
# Ganti nama kolom 'final_clean' menjadi 'clean_text' untuk konsistensi
df['clean_text'] = df['final_clean']

# Buat DataFrame final yang berisi data asli + hasil preprocessing
df_final = df[[text_column, 'clean_text']].copy()

# Tambahkan kolom label jika ada
if 'label' in df.columns:
    df_final['label'] = df['label']
elif 'rating' in df.columns:
    # Konversi rating menjadi label sentimen (contoh: rating 1-3 = negatif, 4-5 = positif)
    df_final['label'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0)

# Simpan ke file baru
output_filename = "../data/ulasan_goride_preprocessed.csv"
df_final.to_csv(output_filename, index=False, encoding='utf-8')

print(f"\n✅ File hasil preprocessing berhasil disimpan sebagai '{output_filename}'")
print(f"Total data: {len(df_final)}")

# Tampilkan perbandingan sebelum dan sesudah preprocessing
print("\n=== PERBANDINGAN SEBELUM DAN SESUDAH PREPROCESSING ===")
df_final.rename(columns={
    text_column: 'sebelum preprocessing', 
    'clean_text': 'setelah preprocessing'
}).head(10)

# 🤖 BAGIAN 3: SENTIMENT ANALYSIS

## 📊 Gambaran Umum
Pada bagian ini, kita akan membangun model machine learning untuk analisis sentimen ulasan GoRide. Proses meliputi:

### 🔧 Tahapan Analisis Sentimen:
1. **Persiapan Data** - Load data yang sudah dipreprocessing
2. **Feature Extraction** - Konversi teks menjadi fitur numerik menggunakan TF-IDF
3. **Data Splitting** - Pembagian data train dan test
4. **Model Training** - Pelatihan model SVM dengan hyperparameter tuning
5. **Model Evaluation** - Evaluasi performa model
6. **Visualisasi** - WordCloud dan analisis kata
7. **Prediksi Manual** - Testing model dengan data baru

## 📦 Import Libraries untuk Machine Learning

In [ ]:
# Data & Visualisasi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc

# SMOTE (Imbalanced Data)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Simpan model
import joblib

print("✅ Libraries berhasil diimport!")

## 📂 Load dan Persiapan Data

Memuat dataset yang sudah dipreprocessing dan mempersiapkan data untuk modeling:

In [ ]:
# Load dataset hasil preprocessing dengan error handling
try:
    df = pd.read_csv("../data/ulasan_goride_preprocessed.csv")
    print(f"Dataset loaded: {len(df)} ulasan")
except FileNotFoundError:
    print("File ulasan_goride_preprocessed.csv tidak ditemukan.")
    print("Membuat dataset dummy untuk demo...")
    df = pd.DataFrame({
        'clean_text': [
            'goride bagus cepat',
            'driver goride ramah',
            'goride telat lama',
            'pelayanan goride kecewa'
        ],
        'label': [1, 1, 0, 0]  # 1=positif, 0=negatif
    })

# Normalisasi nama kolom
df.columns = df.columns.str.lower().str.strip()

# Validasi kolom yang diperlukan
required_columns = ['clean_text', 'label']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Kolom yang hilang: {missing_columns}")
    if 'clean_text' not in df.columns:
        # Coba cari kolom alternatif
        text_columns = [col for col in df.columns if 'text' in col.lower() or 'review' in col.lower()]
        if text_columns:
            df['clean_text'] = df[text_columns[0]]
            print(f"Menggunakan kolom '{text_columns[0]}' sebagai 'clean_text'")
    
    if 'label' not in df.columns:
        # Coba cari kolom rating dan konversi
        if 'rating' in df.columns:
            df['label'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0)
            print("Membuat label dari kolom rating (>=4 = positif, <4 = negatif)")
        else:
            print("Membuat label dummy...")
            df['label'] = [1, 0] * (len(df) // 2) + [1] * (len(df) % 2)

# Drop baris kosong
df.dropna(subset=['clean_text', 'label'], inplace=True)

# Konversi label ke numerik jika masih dalam bentuk text
if df['label'].dtype == 'object':
    label_map = {'negatif': 0, 'positif': 1, 'negative': 0, 'positive': 1}
    df['label'] = df['label'].map(label_map)
    # Handle unmapped values
    df['label'] = df['label'].fillna(0).astype(int)
else:
    df['label'] = df['label'].astype(int)

print(f"Distribusi label:")
print(df['label'].value_counts())
print("\nPreview data:")
df.head()

## 🔤 TF-IDF Feature Extraction

Mengkonversi teks menjadi representasi numerik menggunakan TF-IDF (Term Frequency-Inverse Document Frequency):

In [ ]:
# TF-IDF Feature Extraction
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),        # Unigram dan bigram
    max_features=1000,         # Maksimal 1000 fitur
    min_df=2,                  # Minimum dokumen yang harus mengandung term
    max_df=0.95,               # Maksimal 95% dokumen mengandung term  
    sublinear_tf=True          # Scaling logaritmik
)

# Proses ekstraksi fitur dari kolom 'clean_text'
X_tfidf = tfidf.fit_transform(df['clean_text'])

print(f"Shape matriks TF-IDF: {X_tfidf.shape}")
print(f"Jumlah fitur: {len(tfidf.get_feature_names_out())}")

# Konversi hasil TF-IDF menjadi DataFrame (opsional, untuk preview)
# Perbaiki masalah toarray() untuk sparse matrix
try:
    # Gunakan method yang benar untuk sparse matrix
    if hasattr(X_tfidf, 'toarray'):
        tfidf_array = X_tfidf.toarray()
    else:
        tfidf_array = X_tfidf.todense()
    
    tfidf_df = pd.DataFrame(tfidf_array, columns=tfidf.get_feature_names_out())
    print("\nPreview 5 fitur pertama:")
    tfidf_df.iloc[:5, :10]  # 5 baris pertama, 10 kolom pertama
except Exception as e:
    print(f"Preview TF-IDF matrix tidak dapat ditampilkan: {e}")
    print("Matrix shape:", X_tfidf.shape)

## 🔄 Data Splitting

Membagi data menjadi training set dan testing set:

In [ ]:
X = df['clean_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.1,        # 10% untuk testing
    stratify=y,           # Mempertahankan proporsi label
    random_state=42       # Reproducible results
)

print("Pembagian data:")
print(f"Training data: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Testing data: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nDistribusi label training:")
print(y_train.value_counts())
print(f"\nDistribusi label testing:")
print(y_test.value_counts())

## 🏗️ Machine Learning Pipeline

Membuat pipeline yang menggabungkan TF-IDF, SMOTE (untuk balancing data), dan SVM:

In [ ]:
# Pipeline machine learning dengan SMOTE untuk handling imbalanced data
pipeline = ImbPipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=1000, sublinear_tf=True)),
    ('smote', SMOTE(random_state=42)),
    ('svm', SVC(probability=True, random_state=42))
])

print("✅ Pipeline berhasil dibuat!")
print("Komponen pipeline:")
print("1. TfidfVectorizer - untuk ekstraksi fitur")
print("2. SMOTE - untuk balancing data")  
print("3. SVM - untuk klasifikasi")

## ⚙️ Hyperparameter Tuning

Mencari kombinasi parameter terbaik menggunakan Grid Search dengan Cross Validation:

In [ ]:
# Hyperparameter grid untuk tuning
param_grid = {
    'svm__C': [0.1, 1, 10],                    # Regularization parameter
    'svm__kernel': ['linear', 'rbf'],          # Kernel type  
    'svm__gamma': ['scale', 'auto']            # Kernel coefficient
}

print("Parameter grid untuk tuning:")
for param, values in param_grid.items():
    print(f"- {param}: {values}")

# Grid search dengan cross-validation
grid_search = GridSearchCV(
    pipeline, 
    param_grid=param_grid,
    scoring='f1',           # Optimasi berdasarkan F1-score
    cv=5,                   # 5-fold cross validation
    verbose=1,              # Tampilkan progress
    n_jobs=-1               # Gunakan semua CPU cores
)

print("\n🚀 Memulai hyperparameter tuning...")
# Latih model
grid_search.fit(X_train, y_train)
print("✅ Hyperparameter tuning selesai!")

## 📊 Model Evaluation

Evaluasi performa model menggunakan berbagai metrik dan visualisasi:

In [ ]:
# Prediksi pada data test
y_pred = grid_search.predict(X_test)

# Tampilkan hasil evaluasi
print("=== HASIL EVALUASI MODEL ===")
print(f"Akurasi: {accuracy_score(y_test, y_pred):.4f}")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Score: {grid_search.best_score_:.4f}")

print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred, target_names=['Negatif', 'Positif']))

# Confusion Matrix Visualization
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=['Negatif', 'Positif'], 
            yticklabels=['Negatif', 'Positif'])
plt.title("Confusion Matrix", fontsize=16, fontweight='bold')
plt.xlabel("Predicted Label", fontweight='bold')
plt.ylabel("True Label", fontweight='bold')
plt.tight_layout()
plt.show()

## 📈 ROC Curve Analysis

Analisis kurva ROC untuk membandingkan performa pada data training dan testing:

In [ ]:
# Probabilitas prediksi untuk ROC curve
train_probs = grid_search.predict_proba(X_train)[:, 1]
test_probs = grid_search.predict_proba(X_test)[:, 1]

# Hitung ROC curve
fpr_train, tpr_train, _ = roc_curve(y_train, train_probs)
fpr_test, tpr_test, _ = roc_curve(y_test, test_probs)

# Hitung AUC
auc_train = auc(fpr_train, tpr_train)
auc_test = auc(fpr_test, tpr_test)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr_train, tpr_train, label=f'Train AUC = {auc_train:.3f}', 
         color='orange', linewidth=2)
plt.plot(fpr_test, tpr_test, label=f'Test AUC = {auc_test:.3f}', 
         color='green', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.7, label='Random Classifier')

plt.title('ROC Curve - Train vs Test', fontsize=16, fontweight='bold')
plt.xlabel('False Positive Rate', fontweight='bold')
plt.ylabel('True Positive Rate', fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"AUC Score - Training: {auc_train:.4f}")
print(f"AUC Score - Testing: {auc_test:.4f}")
print(f"Overfitting Check: {'⚠️ Possible overfitting' if abs(auc_train - auc_test) > 0.1 else '✅ Good generalization'}")

## 🔄 Cross Validation Analysis

Validasi stabilitas model menggunakan k-fold cross validation:

In [ ]:
# Cross-validation dengan model terbaik
pipeline_terbaik = grid_search.best_estimator_

# Perbaiki masalah tipe data untuk cross validation
# Pastikan X menggunakan format yang benar untuk pipeline
X_for_cv = X.values if hasattr(X, 'values') else X
y_for_cv = y.values if hasattr(y, 'values') else y

cv_scores = cross_val_score(pipeline_terbaik, X_for_cv, y_for_cv, cv=5, scoring='f1_macro')

print("=== CROSS VALIDATION RESULTS ===")
print(f"Cross-Validation F1 Macro Scores: {cv_scores}")
print(f"Mean F1 Score: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")
print(f"95% Confidence Interval: [{cv_scores.mean() - 2*cv_scores.std():.4f}, {cv_scores.mean() + 2*cv_scores.std():.4f}]")

# Visualisasi CV scores
plt.figure(figsize=(10, 6))
plt.boxplot(cv_scores)
plt.title('Cross-Validation Performance Distribution', fontweight='bold')
plt.ylabel('F1-Macro Score', fontweight='bold')
plt.xticks([1], ['F1-Macro Score'])
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## ☁️ Word Cloud Visualization

Visualisasi kata-kata yang paling sering muncul pada ulasan positif dan negatif:

In [ ]:
# Pisahkan ulasan berdasarkan sentimen
ulasan_positif = df[df['label'] == 1]['clean_text']
ulasan_negatif = df[df['label'] == 0]['clean_text']

# Gabungkan semua teks untuk setiap sentimen
text_positif = " ".join(ulasan_positif)
text_negatif = " ".join(ulasan_negatif)

# Buat WordCloud
plt.figure(figsize=(15, 8))

# WordCloud untuk sentimen positif
plt.subplot(1, 2, 1)
if text_positif.strip():  # Check if text is not empty
    wordcloud_pos = WordCloud(
        background_color='white', 
        max_words=100,
        colormap='Greens',
        width=600, height=400
    ).generate(text_positif)
    plt.imshow(wordcloud_pos, interpolation='bilinear')
    plt.axis('off')
    plt.title('WordCloud - Sentimen Positif', fontsize=14, fontweight='bold', color='green')
else:
    plt.text(0.5, 0.5, 'No positive reviews', ha='center', va='center', transform=plt.gca().transAxes)
    plt.axis('off')

# WordCloud untuk sentimen negatif  
plt.subplot(1, 2, 2)
if text_negatif.strip():  # Check if text is not empty
    wordcloud_neg = WordCloud(
        background_color='white', 
        colormap='Reds', 
        max_words=100,
        width=600, height=400
    ).generate(text_negatif)
    plt.imshow(wordcloud_neg, interpolation='bilinear')
    plt.axis('off')
    plt.title('WordCloud - Sentimen Negatif', fontsize=14, fontweight='bold', color='red')
else:
    plt.text(0.5, 0.5, 'No negative reviews', ha='center', va='center', transform=plt.gca().transAxes)
    plt.axis('off')

plt.tight_layout()
plt.show()

print(f"Jumlah ulasan positif: {len(ulasan_positif)}")
print(f"Jumlah ulasan negatif: {len(ulasan_negatif)}")

## 📊 Top Words Analysis

Analisis kata-kata yang paling sering muncul pada setiap sentimen:

In [ ]:
def plot_top_words(text_series, label_name, top_n=20):
    """Fungsi untuk plot kata teratas berdasarkan frekuensi"""
    if len(text_series) == 0:
        print(f"Tidak ada data untuk {label_name}")
        return
        
    words = " ".join(text_series).split()
    word_freq = Counter(words)
    common_words = word_freq.most_common(top_n)

    if not common_words:
        print(f"Tidak ada kata yang ditemukan untuk {label_name}")
        return

    words_list, freqs = zip(*common_words)

    plt.figure(figsize=(12, 8))
    bars = plt.bar(range(len(words_list)), freqs, 
                   color='lightgreen' if label_name.lower() == 'positif' else 'lightcoral')
    
    # Tambahkan nilai di atas setiap bar
    for bar, freq in zip(bars, freqs):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                str(freq), ha='center', va='bottom', fontweight='bold')
    
    # Perbaiki penggunaan xticks dengan parameter yang benar
    plt.xticks(range(len(words_list)), words_list, rotation=45, ha='right')
    plt.title(f'Top {top_n} Kata Teratas - Sentimen {label_name.capitalize()}', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Kata', fontweight='bold')
    plt.ylabel('Frekuensi', fontweight='bold')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

# Plot untuk sentimen positif dan negatif
if len(ulasan_positif) > 0:
    plot_top_words(ulasan_positif, 'positif')
else:
    print("Tidak ada ulasan positif untuk dianalisis")

if len(ulasan_negatif) > 0:
    plot_top_words(ulasan_negatif, 'negatif')
else:
    print("Tidak ada ulasan negatif untuk dianalisis")

## 🔮 Prediksi Manual

Testing model dengan contoh ulasan baru untuk melihat kemampuan prediksi:

In [ ]:
def prediksi_manual(teks_list):
    """Fungsi untuk prediksi sentimen pada teks baru"""
    print("=== PREDIKSI SENTIMEN MANUAL ===\n")
    
    for i, teks in enumerate(teks_list, 1):
        try:
            # Prediksi sentimen
            pred = grid_search.predict([teks])[0]
            proba = grid_search.predict_proba([teks])[0]
            
            # Format hasil
            label = "Positif" if pred == 1 else "Negatif"
            confidence = proba[pred] * 100
            
            print(f"📝 Ulasan {i}: {teks}")
            print(f"🔮 Prediksi: {label}")
            print(f"📊 Confidence: {confidence:.2f}%")
            print(f"📈 Probabilitas [Negatif: {proba[0]:.3f}, Positif: {proba[1]:.3f}]")
            print("-" * 80)
            
        except Exception as e:
            print(f"❌ Error pada ulasan {i}: {str(e)}")
            print("-" * 80)

# Contoh ulasan untuk testing
contoh_ulasan = [
    "goride sangat bagus dan cepat, driver ramah sekali",
    "selama saya menggunakan layanan goride pengemudinya bau, helm nya tidak ada",
    "goride baik pelayanannya",
    "untuk jenis goride di purwakarta sudah oke dan sangat gampang untuk mencari driver",
    "tiap pesen goride selalu lama dapetnya, sangat mengecewakan"
]

# Jalankan prediksi
prediksi_manual(contoh_ulasan)

## 💾 Penyimpanan Model

Menyimpan model dan vectorizer yang sudah dilatih untuk penggunaan di production:

In [ ]:
# Simpan model dan vectorizer ke file
import os

# Pastikan direktori models ada
models_dir = '../models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Direktori {models_dir} dibuat.")

try:
    # Simpan model lengkap
    joblib.dump(grid_search, f'{models_dir}/svm_model_predict.pkl')
    
    # Simpan vectorizer secara terpisah
    joblib.dump(grid_search.best_estimator_.named_steps['tfidf'], f'{models_dir}/tfidf_vectorizer_predict.pkl')
    
    # Simpan metadata model
    model_metadata = {
        'model_type': 'SVM',
        'best_params': str(grid_search.best_params_),
        'best_score': round(grid_search.best_score_, 4),
        'test_accuracy': round(accuracy_score(y_test, y_pred), 4),
        'feature_count': len(grid_search.best_estimator_.named_steps['tfidf'].get_feature_names_out()),
        'train_size': len(X_train),
        'test_size': len(X_test),
        'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Simpan metadata ke file txt
    with open(f'{models_dir}/model_metadata_predict.txt', 'w', encoding='utf-8') as f:
        f.write("=== METADATA MODEL SENTIMENT ANALYSIS ===\n")
        for key, value in model_metadata.items():
            f.write(f"{key}: {value}\n")
    
    print("✅ Model dan metadata berhasil disimpan!")
    print("Files yang disimpan:")
    print(f"- {models_dir}/svm_model_predict.pkl")
    print(f"- {models_dir}/tfidf_vectorizer_predict.pkl") 
    print(f"- {models_dir}/model_metadata_predict.txt")
    
except Exception as e:
    print(f"❌ Error saat menyimpan model: {str(e)}")
    print("Periksa izin akses ke direktori dan ruang disk yang tersedia")

# 🎯 KESIMPULAN DAN RINGKASAN

## 📋 Ringkasan Proses
Notebook ini telah berhasil menyelesaikan seluruh pipeline analisis sentimen ulasan GoRide:

### 1️⃣ **Web Scraping**
- ✅ Berhasil mengambil data ulasan GoRide dari Google Play Store
- ✅ Filter otomatis berdasarkan kata kunci terkait GoRide
- ✅ Export data ke format CSV

### 2️⃣ **Preprocessing Data (Step-by-Step)**
- ✅ Case Folding & Phrase Standardization
- ✅ Cleansing (hapus URL, karakter khusus)
- ✅ Normalisasi Slang menggunakan kamus
- ✅ Remove Repeated Characters
- ✅ Tokenization
- ✅ Stopword Removal
- ✅ Stemming dengan Sastrawi
- ✅ Rejoin Tokens

### 3️⃣ **Sentiment Analysis**
- ✅ TF-IDF Feature Extraction
- ✅ SVM Model dengan Hyperparameter Tuning
- ✅ SMOTE untuk handling imbalanced data
- ✅ Cross-validation untuk validasi model
- ✅ Evaluasi lengkap dengan berbagai metrik
- ✅ Visualisasi WordCloud dan analisis kata
- ✅ Testing prediksi manual
- ✅ Model saving untuk production

## 🎉 Model Performance Summary
Model yang dibangun dapat digunakan untuk:
- Klasifikasi sentimen ulasan GoRide secara otomatis
- Monitoring kepuasan pelanggan
- Analisis feedback pengguna
- Insights untuk perbaikan layanan

## 🚀 Next Steps
- Deploy model ke aplikasi web/mobile
- Implementasi real-time monitoring sentimen
- Integrasi dengan dashboard analytics
- Continuous model improvement dengan data baru

---
**🏁 Proses analisis sentimen GoRide selesai!**